<a href="https://colab.research.google.com/github/lagocode/20251125_ForBookConnectChallenge/blob/main/book_recommend_ver1_0_5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. 초기 파이썬 라이브러리, 모듈 설치**

In [ ]:
pip install streamlit google-generativeai numpy requests beautifulsoup4

In [ ]:
pip install -U google-generativeai

In [ ]:
pip install pandas openpyxl

In [ ]:
# !npm install localtunnel

In [ ]:
import os

# ==========================================
# [긴급] 네트워크/프록시 설정 강제 초기화
# ==========================================
def unset_proxy():
    proxy_keys = [
        "http_proxy", "https_proxy", "HTTP_PROXY", "HTTPS_PROXY",
        "ALL_PROXY", "all_proxy"
    ]
    print("🧹 네트워크 환경변수 청소 중...", end="")
    for key in proxy_keys:
        if key in os.environ:
            del os.environ[key]
            print(f" [삭제: {key}]", end="")
    print(" 완료! ✅")

unset_proxy()

🧹 네트워크 환경변수 청소 중... 완료! ✅


In [ ]:
# import streamlit as st
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
from datetime import datetime

In [ ]:
import pandas as pd
import re
import urllib
import time
import copy
from google.colab import data_table # 코랩 전용 매직 라이브러리
import time
from google.api_core import retry # 이 모듈이 필요합니다

#**2. 도서 대출 이력 정보 데이터, KDC 6판 코드 엑셀 전처리하기**

In [ ]:
# ==========================================
# 1. 파일 로드 및 설정
# ==========================================
# 코랩의 파일 경로 (예: '/content/파일이름.xlsx')
file_path = "/content/sample_data/book_sample_150.xlsx"

try:
    # 엑셀 파일 읽기
    df = pd.read_excel(file_path)
    print("✅ 데이터 로드 완료")
    print(f"   - 전체 크기: {df.shape}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")

✅ 데이터 로드 완료
   - 전체 크기: (150, 20)


In [ ]:
# ==========================================
# 2. 열 이름 확인
# ==========================================
print("\n📋 [현재 열 목록]")
print(df.columns.tolist())
print(df.head())


📋 [현재 열 목록]
['도서 대여도서관', '도서대여일', '반납예정일', '반납일', '반납연기일', '자료상태', '등록번호', '서명', '저자', '발행자', '청구기호', '반납도서관', '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN', '반납연기횟수', '반납연기 도서관부호']
   도서 대여도서관                도서대여일                반납예정일                  반납일  \
0    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/02/02 09:30:52   
1    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/02/02 09:30:54   
2    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/01/17 11:46:53   
3    121002  2025/01/01 00:00:00  2025/01/31 23:59:59  2025/01/31 00:00:00   
4    126024  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/01/14 00:00:00   

  반납연기일  자료상태          등록번호                                         서명  \
0   NaN     1  CMA000104296                              사랑 사랑 사랑하는 이유   
1   NaN     1  AMA000148390                                    사랑이 다예요   
2   NaN     1  AMA000001129                                     억새풀. 5   
3   NaN     1  BJL000324055  이번 생에 정

In [ ]:

# ==========================================
# 3. 특정 열 누락 (열 삭제)
# ==========================================
# 삭제하고 싶은 열 이름들을 리스트에 적어주세요.
cols_to_drop = ['도서 대여도서관', '도서대여일', '반납예정일', '반납연기일', '발행자', '반납도서관',
                '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN',
                '반납연기횟수', '반납연기 도서관부호']

# errors='ignore': 해당 열이 없어도 에러 없이 넘어감
ndf = df.copy()
ndf = ndf.drop(columns=cols_to_drop, errors='ignore')

print(f"\n🗑️ [열 삭제 완료]")
print(f"   - 삭제 시도한 열: {cols_to_drop}")
print(f"   - 남은 열: {ndf.columns.tolist()}")


🗑️ [열 삭제 완료]
   - 삭제 시도한 열: ['도서 대여도서관', '도서대여일', '반납예정일', '반납연기일', '발행자', '반납도서관', '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN', '반납연기횟수', '반납연기 도서관부호']
   - 남은 열: ['반납일', '자료상태', '등록번호', '서명', '저자', '청구기호']


In [ ]:
# ==========================================
# 4. 특정 열의 텍스트 자르기 (조건부 슬라이싱)
# ==========================================
target_column = '청구기호'

def slice_left_3(text):
    r"""
    . 이나 - 를 찾아 그 인덱스 기준 왼쪽 3글자를 반환하는 함수
    예: '813.6' -> '813', '1234.5' -> '234', 'AB-99' -> 'AB'
    """
    text = str(text)

    # 1. 특수문자 위치 탐색
    idx_dot = text.find('.')
    idx_dash = text.find('-')

    # 2. 유효한 인덱스(찾은 것)만 모으기
    found_indices = [i for i in [idx_dot, idx_dash] if i != -1]

    if found_indices:
        # 3. 가장 먼저 나오는 특수문자 위치 선택
        target_idx = min(found_indices)

        # 4. 해당 위치 기준 왼쪽 3글자 슬라이싱
        # target_idx - 3이 음수면 0(처음)부터 시작
        start_idx = max(0, target_idx - 3)
        return text[start_idx:target_idx]
    else:
        # 5. 특수문자가 없으면 그냥 맨 앞 3글자 반환
        return text[:3]

if target_column in ndf.columns:
    # apply 함수를 사용해 각 셀에 로직 적용
    ndf[target_column] = ndf[target_column].apply(slice_left_3)

    print(f"\n✂️ [글자 수 자르기 완료]")
    print(f"   - 대상 열: '{target_column}'")
    print(f"   - 결과 미리보기:\n{ndf[[target_column]].head()}")
else:
    print(f"\n⚠️ '{target_column}' 열이 데이터에 존재하지 않습니다.")


✂️ [글자 수 자르기 완료]
   - 대상 열: '청구기호'
   - 결과 미리보기:
  청구기호
0  843
1  811
2  813
3  513
4  811


In [ ]:
# ==========================================
# 5. 추가 슬라이싱 (숫자 뒤 띄어쓰기 기준 절삭)
# ==========================================
# 적용할 열 이름을 지정하세요 (테스트용으로 '반납일' 사용)
target_column_space = '반납일'

def slice_digit_and_space(text):
    r"""
    숫자(\d) 다음에 공백(\s)이 오면, 공백부터 그 뒤를 모두 삭제
    예: 'Room 1 A' -> 'Room 1', 'Shelf 22 B' -> 'Shelf 22'
    """
    text = str(text)
    # 정규식: 숫자(\d) 뒤에 공백(\s)이 있는 패턴 검색
    match = re.search(r'\d\s', text)
    if match:
        # match.start()는 숫자의 시작 인덱스
        # 숫자까지는 살려야 하므로 start() + 1 까지 슬라이싱
        return text[:match.start() + 1]
    return text

if target_column_space in df.columns:
    ndf[target_column_space] = ndf[target_column_space].apply(slice_digit_and_space)

    print(f"\n✂️ [슬라이싱 2 완료: 숫자 뒤 공백 기준]")
    print(f"   - 대상 열: '{target_column_space}'")
    print(f"   - 결과 미리보기:\n{ndf[[target_column_space]].head()}")
else:
    print(f"\n⚠️ '{target_column_space}' 열이 데이터에 존재하지 않습니다.")


✂️ [슬라이싱 2 완료: 숫자 뒤 공백 기준]
   - 대상 열: '반납일'
   - 결과 미리보기:
          반납일
0  2025/02/02
1  2025/02/02
2  2025/01/17
3  2025/01/31
4  2025/01/14


In [ ]:
ndf.head()

,반납일,자료상태,등록번호,서명,저자,청구기호
0,2025/02/02,1,CMA000104296,사랑 사랑 사랑하는 이유,므언 티 반 글 ; 제시카 러브 그림 ; 신형건 옮김,843
1,2025/02/02,1,AMA000148390,사랑이 다예요,김용택 사랑시 선집 ; 김선형 그림,811
2,2025/01/17,1,AMA000001129,억새풀. 5,임선영 저,813
3,2025/01/31,1,BJL000324055,이번 생에 정신과는 처음이라 : 정신과 전문의가 말하는 정신과 사용 설명서,닥터 온실 지음,513
4,2025/01/14,1,BNQ000117704,강(江)이 쓰는 시(詩) : 낙동강 연작 제5 시조집,서태수 지음,811


In [ ]:
file_path2 = "/content/sample_data/kdc6list2.xlsx"

try:
    # 엑셀 파일 읽기
    kdc_raw_df = pd.read_excel(file_path2, header=None, names=['기호', '장르명'], dtype={'기호': str})
    print("✅ 데이터 로드 완료")
    print(f"   - 전체 크기: {kdc_raw_df.shape}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path2}")

✅ 데이터 로드 완료
   - 전체 크기: (100, 2)


In [ ]:
kdc_raw_df['기호'] = kdc_raw_df['기호'].astype(str)

In [ ]:
kdc_raw_df

,기호,장르명
0,000,총류
1,010,도서학 서지학
2,020,문헌정보학
3,030,백과사전
4,040,강연집 수필집 연설문집
...,...,...
95,950,남아메리카
96,960,오세아니아 양극지방
97,970,기타
98,980,지리


In [ ]:
import pandas as pd
import re
import os

# ==========================================
# 1. 데이터 준비
# ==========================================

# [A] 사용자의 도서관 대출 데이터 (raw_data)
df_library = pd.DataFrame(ndf)

# [B] 새로 구한 KDC 장르표 로드
df_kdc = pd.DataFrame(kdc_raw_df)

# ==========================================
# 2. KDC 장르 매핑 딕셔너리 생성 (Lookup Table)
# ==========================================
# 기호를 Key로, 장르명을 Value로 변환 (검색 속도 향상)
# 예: {'813': '한국소설', '513': '전기공학/통신'}
kdc_dict = dict(zip(df_kdc['기호'].astype(str), df_kdc['장르명']))

def get_genre_name_advanced(call_number, lookup_dict):
    """
    청구기호(3자리)를 이용해 장르표에서 정확한 장르명을 찾습니다.
    (계층적 조회 로직 적용: 정확히 없으면 상위 분류 검색)
    """
    code = str(call_number).split('.')[0] # 소수점 제거 (혹시 있다면)

    # 1. 정확한 매칭 (Exact Match) - 예: 813 -> 한국소설
    if code in lookup_dict:
        return lookup_dict[code]

    # 2. 십단위 매칭 (Tens Fallback) - 예: 813이 없으면 810(한국문학) 검색
    if len(code) == 3:
        tens_code = code[:2] + "0"
        if tens_code in lookup_dict:
            return lookup_dict[tens_code]

    # 3. 백단위 매칭 (Hundreds Fallback) - 예: 813이 없으면 800(문학) 검색
    if len(code) >= 1:
        hundreds_code = code[0] + "00"
        if hundreds_code in lookup_dict:
            return lookup_dict[hundreds_code]

    return "기타" # 매핑 실패 시

# ==========================================
# 3. 데이터 변환 로직 (Helper Functions)
# ==========================================

def clean_author(text):
    """저자명 정제"""
    text = text.split(';')[0]
    text = re.sub(r'(지음|저|글).*', '', text).strip()
    return text

def generate_initial_keywords(title, genre):
    """제목 단어 + 디테일한 장르명으로 키워드 생성"""
    # 제목에서 특수문자 제거하고 단어 단위로 쪼개기
    words = re.sub(r'[^\w\s]', '', title).split()
    # 장르명이 구체적일수록 벡터 품질이 좋아짐 (예: '한국소설')
    keywords = [genre] + words
    return keywords[:6]

# ==========================================
# 4. 실행: 데이터프레임 -> 라이브러리 DB 리스트
# ==========================================
library_db_list = []

print("🔄 데이터 변환 중...")

for idx, row in df_library.iterrows():
    # A. 고급 장르 매핑 적용
    genre_name = get_genre_name_advanced(row['청구기호'], kdc_dict)

    # B. 저자명 정제
    clean_auth = clean_author(row['저자'])

    # C. 날짜 포맷
    loan_date = row['반납일'].replace('/', '-')

    # D. 키워드 생성
    keywords = generate_initial_keywords(row['서명'], genre_name)

    # E. 결과 조립
    book_item = {
        "id": row['등록번호'],
        "title": row['서명'],
        "author": clean_auth,
        "genre": genre_name,  # 이제 '문학' 대신 '영미소설', '한국시' 등이 들어감
        "class_no": row['청구기호'],
        "last_loan_date": loan_date,
        "keywords": keywords
    }
    library_db_list.append(book_item)

# ==========================================
# 5. 결과 출력
# ==========================================
import json
print("\n### 변환 결과 (복사해서 app.py의 LIBRARY_DB에 붙여넣으세요) ###\n")
print(json.dumps(library_db_list, indent=4, ensure_ascii=False))

# 검증용 출력
print(f"\n[검증] 변환 예시:")
print(f"- 원본 청구기호: {df_library.iloc[0]['청구기호']} -> 변환 장르: {library_db_list[0]['genre']}")

🔄 데이터 변환 중...

### 변환 결과 (복사해서 app.py의 LIBRARY_DB에 붙여넣으세요) ###

[
    {
        "id": "CMA000104296",
        "title": "사랑 사랑 사랑하는 이유",
        "author": "므언 티 반",
        "genre": "영미문학",
        "class_no": "843",
        "last_loan_date": "2025-02-02",
        "keywords": [
            "영미문학",
            "사랑",
            "사랑",
            "사랑하는",
            "이유"
        ]
    },
    {
        "id": "AMA000148390",
        "title": "사랑이 다예요",
        "author": "김용택 사랑시 선집",
        "genre": "한국문학",
        "class_no": "811",
        "last_loan_date": "2025-02-02",
        "keywords": [
            "한국문학",
            "사랑이",
            "다예요"
        ]
    },
    {
        "id": "AMA000001129",
        "title": "억새풀. 5",
        "author": "임선영",
        "genre": "한국문학",
        "class_no": "813",
        "last_loan_date": "2025-01-17",
        "keywords": [
            "한국문학",
            "억새풀",
            "5"
        ]
    },
    {
        "id": "BJL000324055",
        "title": "

#**3. LLM 기반 책 키워드 생성**

1. Google Gemini API(벡터화), Google Books API를 가져와서 '책 키워드(길이: 5)'를 생성합니다.

In [ ]:
# ==========================================
# 0. 설정 및 모델 초기화 (가장 중요)
# ==========================================
try:
    from google.colab import userdata
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = ""

if not API_KEY:
    # 실행 시 입력받거나 여기에 직접 문자열로 입력하세요
    API_KEY = input("Google Gemini API Key를 입력하세요: ").strip()

genai.configure(api_key=API_KEY)

# [수정] model 객체를 전역 변수로 확실하게 생성
try:
    model = genai.GenerativeModel('gemini-2.0-flash')
except Exception as e:
    print(f"⚠️ 모델 초기화 경고: {e}")
    # 구형 라이브러리 대비 Fallback
    model = genai.GenerativeModel('gemini-pro')

In [ ]:
# ==========================================
# 2. Google Books API (알라딘 대체)
# ==========================================

def fetch_google_books_description(book_title):
    """
    Google Books API를 이용해 책의 상세 정보를 가져옵니다. (크롤링 아님, API 사용)
    """
    print(f"\n[Google Books] '{book_title}' 정보 검색 중...")

    # Google Books API 엔드포인트
    api_url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": book_title,
        "maxResults": 1,
        "printType": "books",
        "langRestrict": "ko" # 한국어 결과 우선
    }

    try:
        res = requests.get(api_url, params=params)
        data = res.json()

        if "items" in data and len(data["items"]) > 0:
            book_info = data["items"][0]["volumeInfo"]
            title = book_info.get("title", "제목 없음")
            desc = book_info.get("description", "줄거리 정보가 없습니다.")
            authors = ", ".join(book_info.get("authors", []))

            print(f" -> 찾은 책: {title} ({authors})")
            return desc
        else:
            print(" -> Google Books 결과가 없습니다.")
            return None

    except Exception as e:
        print(f" -> API 오류: {e}")
        return None

# [수정됨] genre 인자 추가하여 프롬프트에 반영
def llm_extract_keywords_from_description(model, book_title, description, genre=None):
    """
    책 줄거리를 분석해 키워드 추출.
    [수정] 5초 타임아웃 설정 -> 5초 넘으면 무조건 실패 처리하고 다음으로 넘어감.
    """
    if not description or description == "줄거리 정보가 없습니다.":
        print(f" -> ⚠️ 줄거리 정보 부재로 키워드 추출 스킵", flush=True)
        return None

    print(f"[LLM Thinking] '{book_title}'의 줄거리를 분석해 키워드 추출 중...", end="", flush=True)

    content_text = description
    avoid_instruction = f"단, 장르명인 '{genre}' 자체는 키워드 목록에 포함하지 마." if genre else ""

    prompt = f"""
    책 제목: {book_title}
    책 줄거리/설명:
    {content_text}

    위 내용을 바탕으로 이 책의 특징을 가장 잘 설명하는 '핵심 키워드' 5개를 추출해줘.
    형용사나 명사 형태로. {avoid_instruction}

    결과는 JSON 리스트로 줘: ["키워드1", "키워드2", "키워드3", "키워드4", "키워드5"]
    """
    try:
        # [핵심 수정] timeout=5 (5초) 추가
        # 재시도 옵션(retry)과 상관없이 5초가 지나면 무조건 에러를 발생시킵니다.
        res = model.generate_content(
            prompt,
            generation_config={"response_mime_type": "application/json"},
            request_options={'timeout': 5.0}
        )
        print(" 완료!", flush=True)
        return json.loads(res.text)
    except Exception as e:
        # 타임아웃 에러도 여기서 잡힙니다.
        print(f"\n -> ❌ 실패 (시간 초과/API 에러): {e}", flush=True)
        return None

# ==========================================
# 3. 벡터 및 유사도 함수
# ==========================================

def get_embedding(text):
    try:
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=text,
            task_type="semantic_similarity"
        )
        return np.array(result['embedding'])
    except Exception as e:
        print(f"❌ 임베딩 오류: {e}")
        return np.zeros(768)

def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm1 * norm2)

# ==========================================
# [수정] 가중치 적용 벡터 로직
# ==========================================

def calculate_kdc_score(code1, code2):
    """
    KDC 청구기호(문자열)를 비교하여 계층적 유사도 점수를 반환
    - code1, code2: 예) "813.6", "800"
    """
    # 비교를 위해 문자열로 변환하고 앞뒤 공백 제거
    c1 = str(code1).strip()
    c2 = str(code2).strip()

    # 길이가 짧은 쪽을 기준으로 비교 (최소 1자리 이상이어야 함)
    min_len = min(len(c1), len(c2))
    if min_len == 0: return 0.0

    # [1단계] 3자리(대분류+중분류+소분류) 일치: 같은 세부 장르 (예: 813 == 813)
    # 보통 KDC는 3자리가 기본이므로 3자리까지 확인
    if len(c1) >= 3 and len(c2) >= 3 and c1[:3] == c2[:3]:
        return 1.5  # 강력한 가중치 (벡터 유사도 1.0보다 훨씬 큼)

    # [2단계] 2자리(대분류+중분류) 일치: 비슷한 장르 (예: 813 vs 811 -> 한국문학 내)
    if len(c1) >= 2 and len(c2) >= 2 and c1[:2] == c2[:2]:
        return 1.0  # 중간 가중치

    # [3단계] 1자리(대분류) 일치: 같은 계열 (예: 813 vs 843 -> 문학)
    if c1[:1] == c2[:1]:
        return 0.5  # 약한 가중치

    # 불일치
    return 0.0

def get_content_vector_only(keywords):
    """
    장르를 제외하고, 오직 '키워드'만으로 벡터를 생성 (API 1회 호출)
    """
    # 키워드를 하나의 문장으로 만듦
    kw_str = ", ".join(keywords) if keywords else ""
    # 프롬프트에 '키워드 특징'임을 명시
    semantic_text = f"책의 핵심 키워드 및 특징: {kw_str}"

    return get_embedding(semantic_text)

def vectorize_library(library_db):
    """전체 도서관 벡터화 (장르 제외, 키워드만)"""
    print("\n[System] 전체 장서 벡터화 시작 (키워드 전용)...")
    vectors = {}
    for i, book in enumerate(library_db):
        print(f"\r -> 진행률: {i+1}/{len(library_db)}", end="")

        # [수정] 키워드만 벡터화하는 함수 호출
        vectors[book['id']] = get_content_vector_only(book['keywords'])

        # 무료 등급 속도 제한 (안전하게 4초 대기)
        time.sleep(4.0)
    print("\n[System] 벡터화 완료! ✅")
    return vectors

# ==========================================
# [수정] 단일 책 벡터 갱신 함수 (get_weighted_vector 제거 -> get_content_vector_only 사용)
# ==========================================
def update_single_book_vector(vectors, book):
    """
    [최적화] 변경된 책 1권의 벡터만 갱신 (API 절약)
    장르 제외, 오직 키워드 내용으로만 벡터 생성
    """
    print(f"\n[System] '{book['title']}'의 정보가 변경되어 벡터를 갱신합니다...")
    # [수정됨] get_weighted_vector 대신 get_content_vector_only 사용
    vectors[book['id']] = get_content_vector_only(book['keywords'])
    print(" -> 갱신 완료 ✅")

# ==========================================
# 4. 추천 로직 함수
# ==========================================

def llm_update_user_keywords(model, current_keywords, new_review):
    print(f"\n[LLM Thinking] 사용자 리뷰 분석 중... ('{new_review}')")

    prompt = f"""
    기존 사용자 관심 키워드: {current_keywords}
    새로운 사용자 리뷰: "{new_review}"

    1. 새 리뷰가 기존 키워드에 없는 새로운 취향이나 중요한 통찰을 포함하고 있는지 0~10점으로 평가하세요.
    2. 점수가 7점 이상이라면, 기존 키워드 중 가장 관련성이 떨어지는 것을 제거하고 새 특징을 반영한 키워드를 추가하세요.

    결과는 반드시 JSON 형식으로 주세요:
    {{ "score": 점수(숫자), "reason": "점수 이유", "updated_keywords": ["키워드1", "키워드2", "키워드3", "키워드4", "키워드5"] }}
    """
    try:
        res = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
        return json.loads(res.text)
    except Exception as e:
        print(f"LLM Error: {e}")
        return {"score": 0, "reason": "Error", "updated_keywords": current_keywords}

def run_recommendation_logic(user_keywords, current_book_info, library_vectors):
    print(f"\n[Algorithm] 하이브리드 추천 실행 (벡터 유사도 + KDC 규칙 점수)...")

    # 1. 사용자 키워드 벡터 생성 (장르 제외)
    user_vector = get_content_vector_only(user_keywords)

    # 현재 책의 청구기호
    current_kdc = current_book_info.get('class_no', '000')

    candidates = []
    for book in LIBRARY_DB:
        if str(book['id']) == str(current_book_info['id']): continue

        # A. 벡터 유사도 (내용 연관성)
        vec_score = cosine_similarity(user_vector, library_vectors.get(book['id']))

        # B. KDC 점수 (장르 연관성)
        book_kdc = book.get('class_no', '000')
        kdc_score = calculate_kdc_score(current_kdc, book_kdc)

        # C. 최종 점수 합산
        final_score = vec_score + kdc_score

        candidates.append({
            "book": book,
            "total_score": final_score,
            "vec_score": vec_score,
            "kdc_score": kdc_score
        })

    # 점수순 정렬
    candidates.sort(key=lambda x: x['total_score'], reverse=True)
    top_3 = candidates[:3]

    final_recommendations = []
    for item in top_3:
        # 추천 사유 생성 logic
        reason = ""
        if item['kdc_score'] >= 5.0: reason = "동일 세부 장르 (Best)"
        elif item['kdc_score'] >= 3.0: reason = "비슷한 장르"
        elif item['kdc_score'] >= 1.0: reason = "같은 계열 도서"
        else: reason = "장르는 다르지만 내용이 비슷함 (Unexpected Discovery)"

        final_recommendations.append({
            "type": "Hybrid Pick",
            "book": item['book'],
            "reason": reason,
            "score": item['total_score'],
            "keywords": item['book']['keywords'],
            "vec": item['vec_score'],  # ✅ [수정] 여기를 추가해야 에러가 사라집니다
            "kdc": item['kdc_score']   # ✅ [수정] 여기를 추가해야 에러가 사라집니다
        })

    return final_recommendations

In [ ]:
def ensure_five_keywords(keywords):
    """[수정됨] 키워드가 5개 미만이면 반복/패딩으로 5개를 맞춤"""
    # 1. 빈 리스트면 기본값
    if not keywords:
        return ["추천도서", "책", "도서", "색다른", "랜덤"]

    # 2. 5개 이상이면 자름
    if len(keywords) >= 5:
        return keywords[:5]

    # 3. 모자르면 채움 (패딩 단어 활용 + 모자르면 마지막 단어 반복)
    padding_pool = ["추천도서", "책", "도서", "색다른", "랜덤"]

    # 기존 키워드 + 패딩 풀
    filled = keywords + padding_pool

    # 그래도 모자르진 않겠지만 안전하게 5개 맞춤
    return filled[:5]

In [ ]:
# [수정 후] - 리스트 객체 그대로 사용 (정상 작동)
LIBRARY_DB = copy.deepcopy(library_db_list)

#**4. 책 키워드 -> 벡터 임베딩 과정**

In [ ]:
# 1. [최적화] 전체 DB 벡터화는 맨 처음에 딱 한 번만 실행함.
# API 호출을 아끼기 위해, 한 번 벡터화한 초기 벡터 데이터는 엑셀로 변환하여 저장해두고, 저장된 엑셀을 불러와서 재사용하여 API 호출 비용을 감소함.

# library_vectors = vectorize_library(LIBRARY_DB)


[System] 전체 장서 벡터화 시작 (키워드 전용)...
 -> 진행률: 150/150
[System] 벡터화 완료! ✅


In [ ]:
#1. [최적화] 전체 DB 벡터화는 맨 처음에 딱 한 번만 실행함.
#전체 DB 벡터화는 api를 사용하기 때문에, 초기 벡터화는 한 번만 시행하고 시연에서는 변환해서 저장해둔 xlsx를 가져와서 초기 벡터 db로 사용함

# print("💾 엑셀 저장을 위해 데이터를 변환 중입니다...")

# # 2. 데이터 변환
# # (Numpy 배열은 엑셀에 바로 저장이 안 되므로, 문자열로 변환해야 합니다)
# excel_data = []
# for book_id, vector in library_vectors.items():
#     excel_data.append({
#         'book_id': book_id,
#         # 벡터 배열을 리스트로 바꾸고 -> JSON 문자열로 변환하여 저장
#         'vector_string': json.dumps(vector.tolist())
#     })

# # 3. 데이터프레임 생성
# df_output = pd.DataFrame(excel_data)

# # 4. 엑셀 파일로 저장
# file_name = "library_vectors.xlsx"
# df_output.to_excel(file_name, index=False)

# print(f"✅ '{file_name}' 파일로 저장 완료! (다운로드하여 시연에 사용하세요)")

💾 엑셀 저장을 위해 데이터를 변환 중입니다...
✅ 'library_vectors.xlsx' 파일로 저장 완료! (다운로드하여 시연에 사용하세요)


In [ ]:
import numpy as np
import os

def load_vectors_from_excel(file_path="/content/sample_data/library_vectors.xlsx"):
    """
    엑셀 파일에 저장된 문자열 형태의 벡터를 읽어와서,
    시스템이 계산 가능한 {book_id: numpy_array} 형태의 딕셔너리로 복원합니다.
    """
    if not os.path.exists(file_path):
        print(f"❌ 오류: '{file_path}' 파일이 없습니다. 먼저 벡터를 생성하고 저장해주세요.")
        return {}

    print(f"📂 '{file_path}'에서 벡터 데이터를 불러오는 중...")

    try:
        # 1. 엑셀 읽기
        df = pd.read_excel(file_path)

        restored_vectors = {}

        for _, row in df.iterrows():
            book_id = row['book_id']
            vector_str = row['vector_string']

            # 2. 문자열(JSON) -> 리스트 -> Numpy 배열로 변환
            # (저장할 때 json.dumps로 풀었으니, json.loads로 다시 묶고 np.array로 만듭니다)
            vector_list = json.loads(vector_str)
            vector_array = np.array(vector_list)

            # 3. 딕셔너리에 저장
            # 주의: 엑셀은 숫자를 자동으로 정수/실수로 변환하므로,
            # LIBRARY_DB의 ID가 문자열이라면 str(book_id)로 변환이 필요할 수 있습니다.
            # 여기서는 있는 그대로 로드합니다.
            restored_vectors[book_id] = vector_array

        print(f"✅ 로딩 완료! 총 {len(restored_vectors)}개의 도서 벡터가 준비되었습니다.")
        return restored_vectors

    except Exception as e:
        print(f"❌ 벡터 로딩 중 오류 발생: {e}")
        return {}

# ==========================================
# [시연용] 사용 예시
# ==========================================
# 시연 시에는 이 함수를 호출해서 변수에 담으세요.
library_vectors = load_vectors_from_excel()

📂 '/content/sample_data/library_vectors.xlsx'에서 벡터 데이터를 불러오는 중...
✅ 로딩 완료! 총 150개의 도서 벡터가 준비되었습니다.


In [ ]:
def inspect_vector_structure(vectors):
    """
    딕셔너리에 담긴 벡터 데이터의 메타 정보(구조, 차원, 타입)를 깔끔하게 출력합니다.
    """
    print("\n🔎 [데이터 구조 분석 보고서]")
    print("=" * 40)

    # 1. 전체 데이터 개수 확인
    total_count = len(vectors)
    print(f"1. 📚 총 도서 개수: {total_count}권")

    if total_count == 0:
        print("   ⚠️ 데이터가 비어있습니다.")
        return

    # 2. 샘플 데이터 하나 꺼내서 분석
    # (딕셔너리의 첫 번째 아이템을 가져옴)
    sample_id = list(vectors.keys())[0]
    sample_vector = vectors[sample_id]

    print(f"\n2. 🔑 Key (도서 ID) 분석")
    print(f"   - 샘플 ID: {sample_id}")
    print(f"   - 데이터 타입: {type(sample_id).__name__}") # str 또는 int

    print(f"\n3. 🔢 Value (임베딩 벡터) 분석")
    print(f"   - 데이터 타입: {type(sample_vector).__name__}") # numpy.ndarray 여야 함

    # Numpy 배열인 경우 상세 정보 출력
    if isinstance(sample_vector, np.ndarray):
        print(f"   - 차원 (Shape): {sample_vector.shape}") # 보통 (768,) 이어야 함
        print(f"   - 내부 데이터형: {sample_vector.dtype}") # float32 또는 float64

        # 벡터 내용 맛보기 (앞 3개, 뒤 2개만 출력)
        preview = f"[{sample_vector[0]:.4f}, {sample_vector[1]:.4f}, ..., {sample_vector[-2]:.4f}, {sample_vector[-1]:.4f}]"
        print(f"   - 값 미리보기: {preview}")
    else:
        print("   ⚠️ 경고: 값이 Numpy 배열이 아닙니다.")

    print("=" * 40)
    print("✅ 검증 완료: 데이터가 정상적인 벡터 구조를 가지고 있습니다." if isinstance(sample_vector, np.ndarray) else "❌ 검증 실패")

# ==========================================
# 사용 예시
# ==========================================
# 위에서 로드한 변수(library_vectors)를 넣으세요
if 'library_vectors' in globals():
    inspect_vector_structure(library_vectors)
else:
    print("library_vectors 변수가 없습니다. 먼저 로드해주세요.")


🔎 [데이터 구조 분석 보고서]
1. 📚 총 도서 개수: 150권

2. 🔑 Key (도서 ID) 분석
   - 샘플 ID: CMA000104296
   - 데이터 타입: str

3. 🔢 Value (임베딩 벡터) 분석
   - 데이터 타입: ndarray
   - 차원 (Shape): (768,)
   - 내부 데이터형: float64
   - 값 미리보기: [-0.0468, 0.0479, ..., 0.0820, -0.0726]
✅ 검증 완료: 데이터가 정상적인 벡터 구조를 가지고 있습니다.


# **5. 서비스 실제 시연 파트**

In [ ]:
# ==========================================
# [NEW] 1단계: 초기화 및 데이터 보강 함수 (Run Once)
# ==========================================
def initialize_system(target_book_title):
    print("=========================================")
    print("      [1단계] 시스템 초기화 및 보강")
    print("=========================================")

    # 1. 책 찾기
    found = [b for b in LIBRARY_DB if b['title'] == target_book_title]
    if not found:
        print(f"❌ 책을 찾을 수 없습니다: {target_book_title}")
        return None, None, None
    current_book = found[0]
    print(f"📖 선택된 책: {current_book['title']}")

    # 2. 구글 API 기반 키워드 보강 (이미 보강된 경우 스킵 가능)
    if not current_book.get('enriched', False):
        desc = fetch_google_books_description(current_book['title'])
        # 장르 코드가 있으면 전달, 없으면 genre 필드 사용
        g_code = current_book.get('class_no', current_book.get('genre'))
        # ✅ 수정된 로직: 줄거리가 없으면 '제목'을 대신 사용
        target_text = book_obj.get('description', '')
        if not target_text:
            print(f" -> ⚠️ 줄거리 정보 부재. 제목('{target_title}')을 기반으로 키워드를 추출합니다.")
            target_text = target_title  # 제목을 분석 대상으로 설정
        new_kws = llm_extract_keywords_from_description(model, current_book['title'], desc, genre=g_code)
        if new_kws is None:
          new_kws = []
        # [중요] 키워드 업데이트: [장르명/코드] + [추출된 키워드]
        # 벡터 생성시엔 내용만 보지만, 리스트엔 장르 정보도 남겨둠
        # 책 정보에 이미 있는 'genre' 이름(경제학)을 맨 앞에 붙임 -> 해결!
        genre_text = current_book.get('genre', '기타')
        current_book['keywords'] = [genre_text] + new_kws
        current_book['enriched'] = True
        print(f" -> 🌟 보강된 키워드: {current_book['keywords']}")
    else:
        print(" -> (이미 보강된 데이터 사용)")

    # 3. 전체 벡터화 (최초 1회)
    # 전역 변수나 외부에서 주입받는 식으로 처리 권장
    # if 'global_vectors' not in globals():
    #     global_vectors = vectorize_library(LIBRARY_DB)
    # else:
    #     print("\n[System] 기존 벡터 데이터 사용")
    #     global_vectors = globals()['global_vectors']

    # 4. 보강된 책만 벡터 갱신
    update_single_book_vector(library_vectors, current_book)

    return current_book, library_vectors

In [ ]:
#모든 책에 대해 구글 책 API를 활용해 텍스트 후기 기반 키워드 업데이트 하기.
#키워드는 llm이 후기를 읽고, 중요하다고 생각하는 키워드들을 뽑아서, 기존의 키워드들과 비교하고 바꿀 가치가 있는 경우에 기존 키워드를 대체함.
def initialize_all_books_in_db(library_db, library_vectors):
    print("=========================================")
    print(f"📚 전체 도서({len(library_db)}권) 데이터 보강 및 벡터 갱신 시작")
    print("=========================================")
    print("⚠️ 주의: 무료 티어 제한을 피하기 위해 천천히 진행됩니다.\n", flush=True)

    success_count = 0
    fail_count = 0
    skip_count = 0

    for i, book in enumerate(library_db):
        print(f"\n[{i+1}/{len(library_db)}] Processing: {book['title']}...", flush=True)

        # 1. 이미 보강된 책은 건너뛰기
        if book.get('enriched', True):
            print(" -> ⏭️ 이미 보강된 데이터입니다. 건너뜁니다.", flush=True)
            skip_count += 1
            continue

        try:
            # 2. Google Books API 호출
            desc = fetch_google_books_description(book['title'])
            time.sleep(1.0)

            # 3. LLM 키워드 추출 시도
            new_kws = llm_extract_keywords_from_description(
                model,
                book['title'],
                desc,
                genre=book.get('genre', '')
            )

            # [예외 처리] None이 반환되면(에러 발생 시)
            if new_kws is None:
                print(" -> ⚠️ (API 에러 감지) 기존 데이터를 유지하고 잠시 쉽니다.", flush=True)
                fail_count += 1

                # 429 에러일 가능성이 높으므로 30초 쿨다운 (화면에 카운트다운 표시)
                print(" -> ⏳ API 과부하 쿨다운: 30초 대기 시작...", end="", flush=True)
                time.sleep(30.0)
                print(" 재개합니다!", flush=True)
                continue

            # 4. 성공 시 데이터 업데이트
            genre_name = book.get('genre', '기타')
            book['keywords'] = [genre_name] + new_kws
            book['enriched'] = True

            print(f" -> 🌟 갱신된 키워드: {book['keywords']}", flush=True)

            # 5. 벡터 갱신
            update_single_book_vector(library_vectors, book)

            success_count += 1

            # [중요] 무료 티어는 분당 15회 제한 -> 성공 후 10초 쉬면 안전 (6회/분)
            print(" -> ⏳ 안전 대기 중 (10초)...", end="", flush=True)
            time.sleep(10.0)
            print(" 완료", flush=True)

        except Exception as e:
            print(f" -> ❌ 알 수 없는 시스템 오류: {e}", flush=True)
            fail_count += 1
            time.sleep(5.0)

    print("\n=========================================")
    print(f"🎉 전체 작업 종료")
    print(f"✅ 성공: {success_count}권")
    print(f"⏭️ 스킵: {skip_count}권")
    print(f"⚠️ 실패: {fail_count}권")
    print("=========================================", flush=True)

    return library_db, library_vectors

# 실행
if __name__ == "__main__":
    LIBRARY_DB, library_vectors = initialize_all_books_in_db(LIBRARY_DB, library_vectors)
# 3. 결과 확인 (앞서 만든 테이블 코드 실행 권장)
print("데이터 갱신이 완료되었습니다. 데이터프레임으로 확인해보세요.")


In [ ]:
#API 출력물 백업 저장 코드
# 작업 결과(DB + 벡터) 백업
# 1. 벡터 저장 (이전과 동일)
excel_data = []
for book_id, vector in library_vectors.items():
    excel_data.append({
        'book_id': book_id,
        'vector_string': json.dumps(vector.tolist())
    })
pd.DataFrame(excel_data).to_excel("library_vectors_enriched22222.xlsx", index=False)

# 2. 책 정보(키워드 포함) 저장
# keywords 리스트를 문자열로 변환해서 저장
db_df = pd.DataFrame(LIBRARY_DB)
db_df['keywords'] = db_df['keywords'].apply(lambda x: json.dumps(x, ensure_ascii=False))
db_df.to_excel("library_db_enriched22222.xlsx", index=False)

print("💾 모든 데이터 백업 완료 (library_vectors_enriched.xlsx, library_db_enriched.xlsx)")

💾 모든 데이터 백업 완료 (library_vectors_enriched.xlsx, library_db_enriched.xlsx)


In [ ]:
#APi 백업을 다시 원래 데이터 형식으로 변환함.
def restore_system_from_backup(db_file="/content/sample_data/library_db_enriched2.xlsx", vec_file="/content/sample_data/library_vectors_enriched2.xlsx"):
    """
    백업된 엑셀 파일 2개를 읽어서 데이터와 벡터를 복원합니다.
    *수정사항: enriched 컬럼의 빈값(NaN)을 0(False)으로 확실하게 처리함
    """
    print(f"[System] 백업 데이터 복원 시작...")

    if not os.path.exists(db_file) or not os.path.exists(vec_file):
        print(f"❌ 오류: 백업 파일이 없습니다. ({db_file}, {vec_file})")
        return [], {}

    # -------------------------------------------------------
    # 1. DB 복원 (LIBRARY_DB)
    # -------------------------------------------------------
    try:
        df_db = pd.read_excel(db_file)

        # [핵심 수정] 'enriched' 컬럼의 NaN(빈칸) 처리
        # NaN은 Python 조건문에서 True로 인식되는 버그가 있음 -> 0(False)으로 채워야 함
        if 'enriched' in df_db.columns:
            df_db['enriched'] = df_db['enriched'].fillna(0) # 빈칸 -> 0
        else:
            df_db['enriched'] = 0 # 컬럼이 아예 없으면 0으로 생성

        # 'keywords' 컬럼 복원 (문자열 -> 리스트)
        if 'keywords' in df_db.columns:
            df_db['keywords'] = df_db['keywords'].apply(
                lambda x: json.loads(x) if isinstance(x, str) else x
            )

        # 데이터프레임 -> 딕셔너리 리스트 변환
        restored_db = df_db.to_dict('records')
        print(f" ✅ DB 복원 완료: {len(restored_db)}권 (enriched 상태 처리됨)")

    except Exception as e:
        print(f"❌ DB 복원 중 오류: {e}")
        return [], {}

    # -------------------------------------------------------
    # 2. 벡터 복원 (library_vectors)
    # -------------------------------------------------------
    try:
        df_vec = pd.read_excel(vec_file)
        restored_vectors = {}

        for _, row in df_vec.iterrows():
            book_id = row['book_id']
            # 엑셀 저장시 ID가 숫자/문자 혼용될 수 있으므로 원본 DB와 타입 매칭 필요할 수 있음
            # 여기서는 파일에 있는 그대로 키로 사용
            vector_str = row['vector_string']

            vector_list = json.loads(vector_str)
            restored_vectors[book_id] = np.array(vector_list)

        print(f" ✅ 벡터 복원 완료: {len(restored_vectors)}개")

    except Exception as e:
        print(f"❌ 벡터 복원 중 오류: {e}")
        return [], {}

    print("🎉 모든 데이터가 메모리로 로드되었습니다.")
    return restored_db, restored_vectors


# ==========================================
# [사용 예시] 메인 코드에서 이렇게 호출하세요
# ==========================================
if __name__ == "__main__":
    # 복원 실행
    LIBRARY_DB, library_vectors = restore_system_from_backup()

    # 확인용 출력
    if LIBRARY_DB:
        print(f"\n[확인] 첫 번째 책 제목: {LIBRARY_DB[0]['title']}")
        print(f"[확인] 첫 번째 책 관련 데이터: {LIBRARY_DB[0]} 리스트여야 함)")
        print(f"[확인] 첫 번째 책 키워드 타입: {type(LIBRARY_DB[0]['keywords'])} (리스트여야 함)")
        print(f"\n[검증] 첫 번째 책 'enriched' 값: {LIBRARY_DB[0].get('enriched')} (0 또는 1이어야 함)")

    if library_vectors:
        first_id = LIBRARY_DB[0]['id']
        if first_id in library_vectors:
            print(f"[확인] 벡터 차원: {library_vectors[first_id].shape} (보통 (768,))")

[System] 백업 데이터 복원 시작...
 ✅ DB 복원 완료: 150권 (enriched 상태 처리됨)
 ✅ 벡터 복원 완료: 150개
🎉 모든 데이터가 메모리로 로드되었습니다.

[확인] 첫 번째 책 제목: 사랑 사랑 사랑하는 이유
[확인] 첫 번째 책 관련 데이터: {'id': 'CMA000104296', 'title': '사랑 사랑 사랑하는 이유', 'author': '므언 티 반', 'genre': '영미문학', 'class_no': 843, 'last_loan_date': '2025-02-02', 'keywords': ['영미문학', '사랑', '관계', '매력', '전략', '성공'], 'enriched': 1} 리스트여야 함)
[확인] 첫 번째 책 키워드 타입: <class 'list'> (리스트여야 함)

[검증] 첫 번째 책 'enriched' 값: 1 (0 또는 1이어야 함)
[확인] 벡터 차원: (768,) (보통 (768,))


In [ ]:
LIBRARY_DB

[{'id': 'CMA000104296',
  'title': '사랑 사랑 사랑하는 이유',
  'author': '므언 티 반',
  'genre': '영미문학',
  'class_no': 843,
  'last_loan_date': '2025-02-02',
  'keywords': ['영미문학', '사랑', '관계', '매력', '전략', '성공'],
  'enriched': 1},
 {'id': 'AMA000148390',
  'title': '사랑이 다예요',
  'author': '김용택 사랑시 선집',
  'genre': '한국문학',
  'class_no': 811,
  'last_loan_date': '2025-02-02',
  'keywords': ['한국문학', '복수', '사랑', '권력', '명예', '계략'],
  'enriched': 1},
 {'id': 'AMA000001129',
  'title': '억새풀. 5',
  'author': '임선영',
  'genre': '한국문학',
  'class_no': 813,
  'last_loan_date': '2025-01-17',
  'keywords': ['한국문학', '잔인한', '흉악한', '냉혹한', '치밀한', '인생'],
  'enriched': 1},
 {'id': 'BJL000324055',
  'title': '이번 생에 정신과는 처음이라 : 정신과 전문의가 말하는 정신과 사용 설명서',
  'author': '닥터 온실',
  'genre': '의학',
  'class_no': 513,
  'last_loan_date': '2025-01-31',
  'keywords': ['의학', '정신과', '정신 건강', '정신과 전문의', '사용 설명서', '자기 이해'],
  'enriched': 1},
 {'id': 'BNQ000117704',
  'title': '강(江)이 쓰는 시(詩) : 낙동강 연작 제5 시조집',
  'author': '서태수',
  'genre':

In [ ]:
library_vectors

{'CMA000104296': array([-5.12437520e-02,  4.95949350e-02, -8.19635500e-03, -3.99979950e-03,
         3.20506900e-02,  2.05282640e-02, -2.18408740e-02,  1.39185670e-02,
         2.53461580e-02, -1.41695850e-02, -8.75459800e-03,  6.36695400e-02,
         3.11924310e-02,  1.19018600e-02,  4.47250940e-02, -1.45177490e-02,
         8.20073900e-02, -8.59249900e-03, -4.04216460e-02, -5.43937200e-02,
        -1.30431980e-02,  2.53905080e-03,  1.39813970e-02, -1.36661940e-02,
        -5.82032870e-02, -4.12354060e-02,  2.02998240e-02, -2.65679800e-03,
         1.38521610e-02, -4.29939780e-02,  5.95977380e-02,  1.30151840e-02,
         2.66236550e-02, -2.20709440e-02,  2.11595220e-02, -1.69949300e-02,
        -3.33797070e-02, -5.94507200e-02,  1.25103550e-02, -4.60170320e-02,
        -5.11659760e-02, -4.03825700e-03, -2.72417760e-02, -6.79272600e-03,
        -6.53407650e-02, -4.46352140e-02, -5.22372800e-02, -2.94462310e-02,
        -7.61433460e-03, -6.93850500e-03,  1.71074250e-02, -8.27449560e-

#1. 후기 기반 갱신 및 책 추천 시스템 - 구현 로직 파트

In [ ]:
#추천사 꾸미는 함수
def llm_write_short_recommendation(model, book_title, keywords, genre, review, technical_reason):
    """
    책의 정보와 선정 이유를 바탕으로 30~40자 내외의 짧은 추천 멘트를 작성
    """
    # 키워드 리스트에서 장르(0번 인덱스)가 섞여 있을 수 있으니, 순수 키워드만 필터링하거나 그대로 사용
    # 여기서는 문자열로 변환
    kw_str = ", ".join(keywords)

    prompt = f"""
    [역할]
    당신은 도서관의 친절하고 센스 있는 사서입니다.

    [정보]
    책 제목: {book_title}
    장르: {genre}
    사용자 리뷰: {review} (이 리뷰의 감정이나 문맥을 반영할 것)
    특징 키워드: {kw_str}
    시스템 선정 이유: {technical_reason}

    [지시 사항]
    위 정보를 바탕으로 사용자에게 이 책을 권하는 한 문장의 추천사를 작성하세요.

    [필수 제약 조건] (반드시 지킬 것)
    1. **절대** 볼드체(**), 이탤릭(*), 따옴표("", '') 등 마크다운 서식을 사용하지 마세요. 오직 순수한 텍스트만 출력하세요.
    2. 말투는 반드시 부드러운 '해요체'(~요, ~입니다)로 통일하세요. (~다, ~한다 같은 어미 절대 금지)
    3. 길이는 공백 포함 60자 이내로 간결하게 작성하세요.
    4. 사용자의 직전 리뷰 내용이 있다면, 그 맥락(공감, 위로, 해결책 등)을 자연스럽게 녹여내세요. 대신, 사용자의 리뷰 중 비속어나 준말과 같이 사서의 격식에 맞지 않는 단어는 사용하지 마세요.
    5. 위 책을 독자에게 추천하는 이유를 '특징 키워드'에 특히 가중치를 많이 두세요.
    6. 단순한 사실 나열보다는 특히 사용자 리뷰와 선정된 책의 를 읽고 "왜 독자에게 추천하는지" 강조하세요.
    7. 그리고, 특징 키워드 5가지를 최대한 전부 활용하려고 시도해보세요. 최대한 특징 키워드들의 내용을 반영하세요.
    """
    try:
        response = model.generate_content(prompt)
        result_text = response.text.strip()

        # [추가된 후처리] 마크다운 및 특수기호 강제 제거
        # 볼드(**), 이탤릭(*), 따옴표(", ') 등을 제거하여 순수 텍스트만 남김
        for char in ['**', '*', '"', "'", '`']:
            result_text = result_text.replace(char, '')

        return result_text

    except Exception as e:
        return f"키워드 {kw_str}가 돋보이는 {genre} 책이라 추천해요!"

In [ ]:
# ==========================================
# [NEW] 2단계: 사용자 인터랙션 루프 (Repeatable) - AI 멘트 적용 버전
# ==========================================
def run_user_feedback_loop(current_book, vectors):
    print("\n=========================================")
    print("      [2단계] 사용자 피드백 및 추천")
    print("=========================================")

    print(f"📘 읽은 책: {current_book['title']}")
    # 키워드가 없는 경우 대비
    current_keywords = current_book.get('keywords', [])
    print(f"🔑 현재 키워드: {current_keywords}")

    review = input("\n💬 후기를 입력하세요 (예: 너무 어렵고 지루했다): ")
    if not review: review = ""

    # -------------------------------------------------------
    # [장르 데이터 보호] 0번 인덱스(장르)는 건드리지 않음
    # -------------------------------------------------------
    if current_keywords:
        fixed_genre = current_keywords[0]       # 0번: 장르 (보호)
        mutable_keywords = current_keywords[1:] # 1번~: 실제 키워드 (수정 대상)
    else:
        fixed_genre = "기타"
        mutable_keywords = []

    # LLM에게는 'mutable_keywords'만 보냄
    res = llm_update_user_keywords(model, mutable_keywords, review)

    if res['score'] >= 7:
        # 장르 + 새 키워드 합치기
        final_kws = [fixed_genre] + res['updated_keywords']
        print(f" -> ✅ 키워드 변경됨: {final_kws}")
    else:
        final_kws = current_keywords
        print(f" -> ⏹️ 키워드 유지됨: {final_kws}")

    # 추천 실행 (알고리즘)
    recs = run_recommendation_logic(final_kws, current_book, vectors)

    print("\n🏆 추천 결과")
    for r in recs:
        book = r['book']

        # [추가된 부분] LLM이 추천 멘트 작성 (API 호출)
        # 키워드 리스트에서 0번(장르 코드)은 제외하고 전달하면 더 자연스러운 문장이 나옴
        clean_keywords = book['keywords'][1:] if len(book['keywords']) > 1 else book['keywords']

        ai_reason = llm_write_short_recommendation(
            model,
            book['title'],
            review,
            clean_keywords,
            book.get('genre', '미상'),
            r['reason']
        )

        print(f"[{r['type']}] {book['title']}")
        print(f" 🗣️ 사서의 한마디: \"{ai_reason}\"")  # AI가 쓴 멘트
        print(f" ⚙️ 시스템 논리: {r['reason']}")      # 원래 로직 이유 (디버깅용)

        # 점수 및 키워드 확인
        vec_score = r.get('vec', 0.0)
        kdc_score = r.get('kdc', 0.0)
        print(f" - 점수: {r['score']:.4f} (Vec {vec_score:.2f} + KDC {kdc_score})")
        print(f" - 키워드: {book['keywords']}")
        print("-" * 30)

#2. 사용자 개인 키워드 기반 책 추천 시스템 - 구현 로직 파트

In [ ]:
# ==========================================
# [Helper] 문자열 벡터 파싱 함수 (기존 동일)
# ==========================================
def parse_vector_string(vec_str):
    """
    "[-0.123, ...]" 문자열을 실제 리스트로 변환
    """
    if isinstance(vec_str, list):
        return vec_str

    try:
        return json.loads(vec_str)
    except:
        try:
            cleaned = vec_str.replace('[', '').replace(']', '').replace('\n', '')
            return [float(x) for x in cleaned.split(',') if x.strip()]
        except Exception as e:
            return []

# ==========================================
# [Algorithm] 키워드 리스트 기반 추천 로직
# ==========================================
def run_recommendation_by_keyword_list(target_vector, library_vectors):
    print(f"\n[Algorithm] 키워드 벡터 매칭 실행...")
    print("   -> 🚨 필터 조건: 키워드 5개 이상인 책만 우선 추천")

    candidates = []

    # 1. 데이터 형태 정규화 (List/Dict/DataFrame 대응)
    iterator = []
    data_mode = "unknown"

    if hasattr(library_vectors, 'to_dict'):
        iterator = library_vectors.to_dict(orient='records')
        data_mode = "list_of_dicts"
    elif isinstance(library_vectors, list):
        iterator = library_vectors
        data_mode = "list_of_dicts"
    elif isinstance(library_vectors, dict):
        first_val = next(iter(library_vectors.values())) if library_vectors else None
        if isinstance(first_val, dict):
            iterator = list(library_vectors.values())
            data_mode = "list_of_dicts"
        else:
            iterator = list(library_vectors.items())
            data_mode = "dict_mapping"
    else:
        return []

    # 2. 전체 탐색 루프
    for item in iterator:
        book_id = None
        book_vector = None

        # 데이터 추출
        if data_mode == "dict_mapping":
            book_id = str(item[0])
            raw_vec = item[1]
            book_vector = parse_vector_string(raw_vec) if isinstance(raw_vec, str) else raw_vec
        elif data_mode == "list_of_dicts":
            if not isinstance(item, dict): continue
            book_id = str(item.get('book_id'))
            vec_str = item.get('vector_string') or item.get('embedding') or item.get('vector')
            if not vec_str: continue
            book_vector = parse_vector_string(vec_str)

        if book_vector is None or len(book_vector) == 0: continue

        # 벡터 유사도 계산
        try:
            vec_score = cosine_similarity(target_vector, book_vector)
        except:
            continue

        # 책 정보 매칭
        found_book = None
        for b in LIBRARY_DB:
            if str(b.get('id')) == book_id:
                found_book = b
                break

        if not found_book: continue

        candidates.append({
            "book": found_book,
            "total_score": vec_score,
            "vec_score": vec_score,
            "reason": f"키워드 유사도 {vec_score:.2f}"
        })

    if not candidates:
        print("❌ 매칭되는 책이 없습니다.")
        return []

    # 3. 점수 정렬
    candidates.sort(key=lambda x: x['total_score'], reverse=True)

    # 4. 필터링 (키워드 5개 이상)
    final_top_3 = []
    for cand in candidates:
        book = cand['book']
        keywords = book.get('keywords', [])

        # 키워드 개수 조건
        if len(keywords) < 5:
            continue

        final_top_3.append(cand)
        if len(final_top_3) == 3:
            break

    # 조건 만족하는게 없으면 그냥 상위 3개 반환 (Fallback)
    if not final_top_3:
        print("⚠️ 키워드 5개 조건을 만족하는 책이 없어, 점수순으로 반환합니다.")
        final_top_3 = candidates[:3]

    # 5. 결과 반환
    final_results = []
    for i, item in enumerate(final_top_3, 1):
        final_results.append({
            "type": f"키워드매칭 {i}위",
            "book": item['book'],
            "reason": item['reason'],
            "score": item['total_score'],
            "vec": item['vec_score']
        })

    return final_results

# ==========================================
# [Main] 사용자 키워드 입력 루프
# ==========================================
def run_keyword_search_loop(vectors, model):
    print("\n=========================================")
    print("      [4단계] 키워드 리스트 직접 검색")
    print("=========================================")

    # 1. 사용자 입력 받기
    print("\n💬 검색할 키워드들을 입력하세요 (공백이나 쉼표로 구분)")
    print("   예시: 우주 과학 미래 로봇 / 사랑, 이별, 슬픔")

    raw_input = input(">>> ")
    if not raw_input.strip():
        print("⚠️ 입력이 없어 종료합니다.")
        return

    # 2. 입력 문자열을 깨끗한 리스트로 변환
    # 쉼표(,)를 공백으로 바꾸고, 공백 기준으로 자름
    user_keywords = [w.strip() for w in raw_input.replace(',', ' ').split() if w.strip()]

    print(f"🔍 입력 키워드 리스트: {user_keywords}")

    if not user_keywords:
        print("❌ 유효한 키워드가 없습니다.")
        return

    # 3. 키워드 리스트를 하나의 문장으로 합쳐서 벡터화
    # (Embedding 모델은 문장 형태의 입력을 잘 처리하므로 띄어쓰기로 합침)
    keywords_str = " ".join(user_keywords)
    target_vector = get_embedding(keywords_str)

    if target_vector is None or len(target_vector) == 0:
        print("❌ 벡터 생성 실패.")
        return

    # 4. 추천 실행
    recs = run_recommendation_by_keyword_list(target_vector, vectors)

    print("\n🏆 추천 결과")
    for r in recs:
        book = r['book']

        # 5. LLM 멘트 작성
        # (책 제목, 사용자 키워드 뭉치, 책 키워드, 선정 이유 전달)
        raw_kws = book.get('keywords', [])
        clean_book_keywords = raw_kws[1:] if len(raw_kws) > 1 else raw_kws

        # 입력 키워드를 '사용자가 원하는 내용'으로 간주하고 넘김
        user_intent = f"키워드({', '.join(user_keywords)})에 맞는 책"

        ai_reason = llm_write_short_recommendation_about_input(
            model,
            clean_book_keywords, # 책의 키워드
            user_intent,         # 사용자의 의도 (입력한 키워드들)
            r['reason']          # 시스템 논리
        )

        print(f"[{r['type']}] {book['title']}")
        print(f" 🗣️ 사서의 한마디: \"{ai_reason}\"")
        print(f" ⚙️ 시스템 논리: {r['reason']}")
        print(f" - 유사도 점수: {r['score']:.4f}")
        print(f" - 책 키워드: {book.get('keywords')}")
        print("-" * 30)

#3. 사용자 줄글 입력 기반 책 추천 시스템 - 구현 로직 파트

In [ ]:
def llm_write_short_recommendation_about_input(model, keywords, need_input, technical_reason):
    """
    책의 정보와 선정 이유를 바탕으로 30~40자 내외의 짧은 추천 멘트를 작성
    """
    # 리스트를 문자열로 변환
    kw_str = ", ".join(keywords)

    prompt = f"""
    [역할]
    당신은 도서관의 친절하고 센스 있는 사서입니다.

    [정보]
    추천된 책 키워드들: {kw_str}
    사용자가 원하는 책: {need_input} (이 리뷰의 감정이나 문맥을 반영할 것)
    시스템 선정 이유: {technical_reason}

    [지시 사항]
    위 정보를 바탕으로 사용자에게 이 책을 권하는 한 문장의 추천사를 작성하세요.

    [필수 제약 조건] (반드시 지킬 것)
    1. **절대** 볼드체(**), 이탤릭(*), 따옴표("", '') 등 마크다운 서식을 사용하지 마세요. 오직 순수한 텍스트만 출력하세요.
    2. 말투는 반드시 부드러운 '해요체'(~요, ~입니다)로 통일하세요. (~다, ~한다 같은 어미 절대 금지)
    3. 길이는 공백 포함 60자 이내로 간결하게 작성하세요.
    4. 사용자가 원하는 책이 있다면, 그 맥락(공감, 위로, 해결책 등)을 자연스럽게 녹여내세요. 대신, 비속어나 줄임말은 쓰지 마세요.
    5. 특징 키워드들을 자연스럽게 문장에 녹여내세요.
    """
    try:
        response = model.generate_content(prompt)
        result_text = response.text.strip()

        # [후처리] 특수기호 제거
        for char in ['**', '*', '"', "'", '`']:
            result_text = result_text.replace(char, '')

        return result_text

    except Exception as e:
        return f"요청하신 '{need_input}' 느낌에 가장 가까운 책이라 추천드려요."

In [ ]:
# 1. 키워드 추출 함수
def llm_extract_keywords_from_input(model, user_input):
    """
    사용자의 줄글 입력(user_input)을 분석하여 검색에 용이한 5개의 핵심 키워드 리스트로 반환
    """
    prompt = f"""
    [지시]
    사용자의 입력 문장에서 핵심적인 검색 키워드 5개를 추출해줘.

    [입력 문장]
    "{user_input}"

    [제약]
    1. 불용어(조사, 어미)를 제거하고 명사 위주로 추출할 것.
    2. 문장의 감정이나 의도(예: 슬픈, 힐링, 공부)도 키워드로 포함할 것.
    3. 정확히 단어 5개를 공백으로 구분해서 한 줄로 출력해. (예: 감동 소설 눈물 위로 가족)
    4. 부가적인 설명이나 특수기호 없이 오직 단어 5개만 출력해.
    """
    try:
        response = model.generate_content(prompt)
        text = response.text.strip()
        keywords = text.split()
        return keywords[:5] # 안전하게 5개까지만 자름
    except Exception as e:
        print(f"⚠️ 키워드 추출 실패: {e}")
        return user_input.split()[:5]

# 2. 벡터 파싱 헬퍼 함수
def parse_vector_string(vec_str):
    """
    "[-0.123, ...]" 문자열을 실제 리스트로 변환
    """
    if isinstance(vec_str, list):
        return vec_str

    try:
        return json.loads(vec_str)
    except:
        try:
            cleaned = vec_str.replace('[', '').replace(']', '').replace('\n', '')
            return [float(x) for x in cleaned.split(',') if x.strip()]
        except Exception as e:
            return []

def run_recommendation_logic_by_input(target_vector, library_vectors):
    print(f"\n[Algorithm] 사용자 입력 기반 벡터 검색 실행...")
    print("   -> 🚨 조건 추가: 키워드가 5개 이상인 책만 추천합니다.")

    candidates = []

    # ----------------------------------------------------
    # [Data Normalization] 데이터 형태 통일
    # ----------------------------------------------------
    iterator = []
    data_mode = "unknown"

    if hasattr(library_vectors, 'to_dict'):
        iterator = library_vectors.to_dict(orient='records')
        data_mode = "list_of_dicts"
    elif isinstance(library_vectors, list):
        iterator = library_vectors
        data_mode = "list_of_dicts"
    elif isinstance(library_vectors, dict):
        first_val = next(iter(library_vectors.values())) if library_vectors else None
        if isinstance(first_val, dict):
            iterator = list(library_vectors.values())
            data_mode = "list_of_dicts"
        else:
            iterator = list(library_vectors.items())
            data_mode = "dict_mapping"
    else:
        return []

    # ----------------------------------------------------
    # [Search Loop] 모든 후보군 점수 계산
    # ----------------------------------------------------
    for item in iterator:
        # 1. ID 및 벡터 추출
        book_id = None
        book_vector = None

        if data_mode == "dict_mapping":
            book_id = str(item[0])
            raw_vec = item[1]
            book_vector = parse_vector_string(raw_vec) if isinstance(raw_vec, str) else raw_vec

        elif data_mode == "list_of_dicts":
            if not isinstance(item, dict): continue
            book_id = str(item.get('book_id'))
            vec_str = item.get('vector_string') or item.get('embedding') or item.get('vector')
            if not vec_str: continue
            book_vector = parse_vector_string(vec_str)

        if book_vector is None or len(book_vector) == 0: continue

        # 2. 벡터 유사도 계산
        try:
            vec_score = cosine_similarity(target_vector, book_vector)
        except:
            continue

        # 3. 책 상세 정보 매칭
        found_book = None
        for b in LIBRARY_DB:
            if str(b.get('id')) == book_id:
                found_book = b
                break

        if not found_book: continue

        # -----------------------------------------------------------
        # [Critical] 여기서 바로 거르지 않고, 일단 후보군에 다 넣음
        # (점수순 정렬 후 상위권에서 5개 조건을 만족하는 놈을 뽑기 위해)
        # -----------------------------------------------------------
        candidates.append({
            "book": found_book,
            "total_score": vec_score,
            "vec_score": vec_score,
            "reason": f"입력 내용과 의미 유사도 {vec_score:.2f}"
        })

    if not candidates:
        print("❌ 유사한 책을 찾지 못했습니다.")
        return []

    # 4. 점수순 정렬 (유사도 높은 순)
    candidates.sort(key=lambda x: x['total_score'], reverse=True)

    # -----------------------------------------------------------
    # [Logic Update] 키워드 5개 이상인 책만 골라서 Top 3 선정
    # -----------------------------------------------------------
    final_top_3 = []

    for cand in candidates:
        book = cand['book']
        keywords = book.get('keywords', [])

        # ✅ 조건 확인: 키워드가 5개 미만이면 스킵 (Next Candidate)
        if len(keywords) < 5:
            continue

        # 조건을 통과했으면 결과 리스트에 추가
        final_top_3.append(cand)

        # 3권이 채워지면 중단
        if len(final_top_3) == 3:
            break

    # 결과가 없을 경우 (모든 책이 키워드 5개 미만인 경우)
    if not final_top_3:
        print("⚠️ 검색 결과는 있으나 '키워드 5개' 조건을 만족하는 책이 없습니다.")
        # (옵션) 비상시에는 그냥 점수 높은거 3개 줄 수도 있음. 여기선 빈 리스트 반환.
        return []

    # 5. 결과 포맷팅
    final_results = []
    for i, item in enumerate(final_top_3, 1):
        final_results.append({
            "type": f"AI 검색 {i}위",
            "book": item['book'],
            "reason": item['reason'],
            "score": item['total_score'],
            "vec": item['vec_score']
        })

    return final_results

# 4. 메인 실행 루프
def run_user_feedback_by_input_loop(vectors, model):
    print("\n=========================================")
    print("      [3단계] 사용자 줄글 입력 -> 키워드 변환 -> 추천")
    print("=========================================")

    need_input = input("\n💬 원하는 내용을 입력하세요 (예: 눈물나게 감동적인 소설 추천해줘): ")
    if not need_input.strip():
        print("⚠️ 입력 내용이 없어 추천을 건너뜁니다.")
        return

    print(f"🔍 입력 내용 분석 중: '{need_input}'")

    # 1. 키워드 변환
    extracted_keywords = llm_extract_keywords_from_input(model, need_input)
    print(f" -> 🔑 추출된 키워드: {extracted_keywords}")

    # 2. 벡터화 (get_embedding은 외부에 정의되어 있어야 함)
    keywords_str = " ".join(extracted_keywords)
    target_vector = get_embedding(keywords_str)

    if target_vector is None or len(target_vector) == 0:
        print("❌ 벡터 생성에 실패했습니다.")
        return

    # 3. 추천 실행
    recs = run_recommendation_logic_by_input(target_vector, vectors)

    print("\n🏆 추천 결과")
    for r in recs:
        book = r['book']

        # 4. LLM 멘트 작성
        raw_kws = book.get('keywords', [])
        clean_book_keywords = raw_kws[1:] if len(raw_kws) > 1 else raw_kws

        ai_reason = llm_write_short_recommendation_about_input(
            model,
            clean_book_keywords,
            need_input,
            r['reason']
        )

        print(f"[{r['type']}] {book['title']}")
        print(f" 🗣️ 사서의 한마디: \"{ai_reason}\"")
        print(f" ⚙️ 시스템 논리: {r['reason']}")
        print(f" - 유사도 점수: {r['score']:.4f}")
        print(f" - 책 키워드: {book.get('keywords')}")
        print("-" * 30)

#**6. 서비스 시연 예제**

1. 후기를 입력 받아 기존 키워드 목록을 갱신함.
2. 갱신된 키워드 목록을 바탕으로 가장 근사한 책 세 권을 llm의 문구로 추천함.

In [ ]:
# ==========================================
# 5. 메인 실행 (Main) - 셀 분리 가능 구조
# ==========================================
if __name__ == "__main__":
    # 1. 초기화 및 보강 (이 부분만 따로 실행 가능)
    target_title = "랑과 나의 사막"
    book_obj, lib_vecs = initialize_system(target_title)

    # 2. 루프 실행 (이 부분만 반복 실행 가능)
    if book_obj and lib_vecs:
        run_user_feedback_loop(book_obj, lib_vecs)

      [1단계] 시스템 초기화 및 보강
📖 선택된 책: 랑과 나의 사막
 -> (이미 보강된 데이터 사용)

[System] '랑과 나의 사막'의 정보가 변경되어 벡터를 갱신합니다...
 -> 갱신 완료 ✅

      [2단계] 사용자 피드백 및 추천
📘 읽은 책: 랑과 나의 사막
🔑 현재 키워드: ['한국문학', '로봇', '사막', '상실', '여정', '애도']

💬 후기를 입력하세요 (예: 너무 어렵고 지루했다): 눈물 흘리고 싶은 분들께 강추합니다

[LLM Thinking] 사용자 리뷰 분석 중... ('눈물 흘리고 싶은 분들께 강추합니다')
 -> ✅ 키워드 변경됨: ['한국문학', '로봇', '사막', '여정', '눈물', '애도']

[Algorithm] 하이브리드 추천 실행 (벡터 유사도 + KDC 규칙 점수)...

🏆 추천 결과
[Hybrid Pick] 억새풀. 5
 🗣️ 사서의 한마디: "한국 문학을 좋아하시는 분들께, 눈물 흘리고 싶을 때 냉혹하지만 치밀한 인생의 억새풀을 강력 추천합니다."
 ⚙️ 시스템 논리: 같은 계열 도서
 - 점수: 2.5000 (Vec 1.00 + KDC 1.5)
 - 키워드: ['한국문학', '잔인한', '흉악한', '냉혹한', '치밀한', '인생']
------------------------------
[Hybrid Pick] 딸에 대하여 : 김혜진 장편소설
 🗣️ 사서의 한마디: "한국문학을 좋아하시는 분이라면, 눈물 흘리고 싶은 분들께 강력 추천하는, 혐오와 배제 속 가족의 의미를 생각하게 하는 성장 소설입니다."
 ⚙️ 시스템 논리: 같은 계열 도서
 - 점수: 2.5000 (Vec 1.00 + KDC 1.5)
 - 키워드: ['한국문학', '레즈비언', '가족', '혐오', '배제', '성장']
------------------------------
[Hybrid Pick] 한국이 싫어서 : 장강명 장편소설
 🗣️ 사서의 한마디: "한국 문학을 좋아하시는 분이라면, 눈물 흘리고 싶을 때 이

In [ ]:
# ==========================================
# 5. 메인 실행 (Main) - 셀 분리 가능 구조
# ==========================================
if __name__ == "__main__":
    # 1. 초기화 및 보강 (이 부분만 따로 실행 가능)
    target_title = "감정의 이해"
    book_obj, lib_vecs = initialize_system(target_title)

    # 2. 루프 실행 (이 부분만 반복 실행 가능)
    if book_obj and lib_vecs:
        run_user_feedback_loop(book_obj, lib_vecs)

      [1단계] 시스템 초기화 및 보강
📖 선택된 책: 감정의 이해
 -> (이미 보강된 데이터 사용)

[System] '감정의 이해'의 정보가 변경되어 벡터를 갱신합니다...
 -> 갱신 완료 ✅

      [2단계] 사용자 피드백 및 추천
📘 읽은 책: 감정의 이해
🔑 현재 키워드: ['심리학', '감정', '탈정서화', '정서적 회복력', '감정 조절', '정서적 경험']

💬 후기를 입력하세요 (예: 너무 어렵고 지루했다): 내가 이해하지 못했던 다른 사람이 이해가 되기 시작했다

[LLM Thinking] 사용자 리뷰 분석 중... ('내가 이해하지 못했던 다른 사람이 이해가 되기 시작했다')
 -> ✅ 키워드 변경됨: ['심리학', '감정', '탈정서화', '정서적 회복력', '감정 조절', '공감 능력']

[Algorithm] 하이브리드 추천 실행 (벡터 유사도 + KDC 규칙 점수)...

🏆 추천 결과
[Hybrid Pick] 히든 포텐셜  : 성공을 이루는 숨은 잠재력의 과학
 🗣️ 사서의 한마디: "심리학 분야에 관심을 보이셨으니, 미처 이해하지 못했던 다른 사람을 이해하고 숨은 잠재력을 발견하는 데 도움을 줄 책입니다."
 ⚙️ 시스템 논리: 같은 계열 도서
 - 점수: 2.4960 (Vec 1.00 + KDC 1.5)
 - 키워드: ['심리학', '히든', '포텐셜', '성공을', '이루는', '숨은']
------------------------------
[Hybrid Pick] 나 혼자만 알고 싶은 실전 심리학
 🗣️ 사서의 한마디: "심리학 리뷰를 남기셨군요, 이 책은 이제껏 이해 못 했던 다른 사람을 이해하게 해 줄 시작점이 될 거예요."
 ⚙️ 시스템 논리: 같은 계열 도서
 - 점수: 1.9960 (Vec 1.00 + KDC 1.0)
 - 키워드: ['심리학', '인간관계', '실생활', '심리', '활용', '사례']
------------------------------
[Hybrid Pick] 나는 나를 바꾸

2. 사용자 개인의 키워드를 벡터화하여 가장 가까운 책 3권을 추천함.

In [ ]:
# 실행 예시
run_keyword_search_loop(library_vectors, model)


      [4단계] 키워드 리스트 직접 검색

💬 검색할 키워드들을 입력하세요 (공백이나 쉼표로 구분)
   예시: 우주 과학 미래 로봇 / 사랑, 이별, 슬픔
>>> 잠잠한 침착한 섬세한 공학 이론 수학
🔍 입력 키워드 리스트: ['잠잠한', '침착한', '섬세한', '공학', '이론', '수학']

[Algorithm] 키워드 벡터 매칭 실행...
   -> 🚨 필터 조건: 키워드 5개 이상인 책만 우선 추천

🏆 추천 결과
[키워드매칭 1위] 이처럼 사소한 것들
 🗣️ 사서의 한마디: "섬세한 시선으로 내면의 갈등을 잠잠히 다독이는, 깊이 있는 성찰을 담은 책을 권해 드려요."
 ⚙️ 시스템 논리: 키워드 유사도 0.91
 - 유사도 점수: 0.9057
 - 책 키워드: ['영미문학', '도덕적 동요', '내적 갈등', '섬세한 관찰', '정교한 문체', '실존적 고민']
------------------------------
[키워드매칭 2위] 딕테=DICTEE
 🗣️ 사서의 한마디: "디지털 소통 속에서 잠잠하고 섬세한 관계를 위한 이론적 고찰을 담은 책을 추천해 드려요."
 ⚙️ 시스템 논리: 키워드 유사도 0.90
 - 유사도 점수: 0.9044
 - 책 키워드: ['영미문학', '공동체 의식', '인간관계', '사회 관계', '디지털 소통', '인류 파편화']
------------------------------
[키워드매칭 3위] 세컨더리 마인드 : 무너질수록 더 강해지는 제2의 정신력
 🗣️ 사서의 한마디: "섬세한 이론과 수학적 사고를 통해 잠잠한 정신력을 키우고 싶다면 이 책을 추천해요."
 ⚙️ 시스템 논리: 키워드 유사도 0.90
 - 유사도 점수: 0.9039
 - 책 키워드: ['윤리학 도덕철학', '세컨더리 마인드', '정신력', '실패', '성장', '시행착오']
------------------------------


3. 사용자가 입력한 줄글의 내용에서 키워드를 선정하고, 가까운 책 세 권을 추천함.

In [ ]:
# 실행 (이 코드가 실행되려면 아래 변수/함수들이 미리 정의되어 있어야 합니다)
# 1. LIBRARY_DB (전체 책 리스트)
# 2. cosine_similarity (함수)
# 3. get_embedding (함수)
# 4. llm_write_short_recommendation_about_input (함수)

run_user_feedback_by_input_loop(library_vectors, model)


      [3단계] 사용자 줄글 입력 -> 키워드 변환 -> 추천

💬 원하는 내용을 입력하세요 (예: 눈물나게 감동적인 소설 추천해줘): 마음껏 휴식하고 싶을 때 즐길 책
🔍 입력 내용 분석 중: '마음껏 휴식하고 싶을 때 즐길 책'
 -> 🔑 추출된 키워드: ['휴식', '책', '즐거움', '마음', '위로']

[Algorithm] 사용자 입력 기반 벡터 검색 실행...
   -> 🚨 조건 추가: 키워드가 5개 이상인 책만 추천합니다.

🏆 추천 결과
[AI 검색 1위] 이처럼 사소한 것들
 🗣️ 사서의 한마디: "마음의 휴식이 필요할 때, 섬세한 문체로 내적 갈등을 깊이 성찰하는 책이 깊은 공감을 드릴 거예요."
 ⚙️ 시스템 논리: 입력 내용과 의미 유사도 0.90
 - 유사도 점수: 0.9007
 - 책 키워드: ['영미문학', '도덕적 동요', '내적 갈등', '섬세한 관찰', '정교한 문체', '실존적 고민']
------------------------------
[AI 검색 2위] 딕테=DICTEE
 🗣️ 사서의 한마디: "디지털 소통 속 파편화된 관계를 돌아보며, 따뜻한 공동체 의식을 느끼고 잠시 쉬어가세요."
 ⚙️ 시스템 논리: 입력 내용과 의미 유사도 0.90
 - 유사도 점수: 0.8995
 - 책 키워드: ['영미문학', '공동체 의식', '인간관계', '사회 관계', '디지털 소통', '인류 파편화']
------------------------------
[AI 검색 3위] 세컨더리 마인드 : 무너질수록 더 강해지는 제2의 정신력
 🗣️ 사서의 한마디: "실패와 시행착오 속에서도 세컨더리 마인드를 통해 성장하는 정신력을 기르며 편안한 휴식을 얻어보세요."
 ⚙️ 시스템 논리: 입력 내용과 의미 유사도 0.90
 - 유사도 점수: 0.8993
 - 책 키워드: ['윤리학 도덕철학', '세컨더리 마인드', '정신력', '실패', '성장', '시행착오']
------------------------------
